In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

if not os.getcwd().endswith('frm2'):
    os.chdir('..')

cwd = os.getcwd()
print(cwd)

In [ ]:
# Read in the data
csv_path = "results/.csv/rab6-compartment-integrity.csv"
data = pd.read_csv(csv_path)


In [ ]:
# Create DataFrame
df = pd.DataFrame(data)
# Rename treatment column header to 'Treatment'
df = df.rename(columns={'treatment': 'Treatment'})

# Rename treatment values
df['Treatment'] = df['Treatment'].replace({'minusiaa': 'Control', 'plusiaa': 'FRM2-KD'})
df

In [ ]:
# Calculate the percentage of True vs False for 'intact?' for each Treatment
percentage_df = df.groupby('Treatment')['intact?'].value_counts(normalize=True).unstack() * 100
percentage_df = percentage_df.round(2)

# Rename columns to ['Fragmented', 'Intact']
percentage_df.columns = ['Fragmented', 'Intact']

# Replace NaN with 0
percentage_df = percentage_df.fillna(0)

# Reorder columns to have 'Intact' first
percentage_df = percentage_df[['Intact', 'Fragmented']]

# Ensure `Treatment` is part of the data and calculate the total count (n) of each treatment group from the original DataFrame
total_count_df = df['Treatment'].value_counts()

# Combine percentage and total count dataframes
result_df = percentage_df.join(total_count_df.rename('Number of vacuoles'))

# Display result
print(result_df)

# Style the DataFrame
styled_df = result_df.style.format({
    'Intact': '{:.2f}%',
    'Fragmented': '{:.2f}%',
    'Number of vacuoles': '{:,.0f}'
}).set_caption("Table: Percentage of Intact and Fragmented Vacuoles per Treatment") \
  .set_table_styles([
      {'selector': 'caption', 'props': [('font-size', '14pt'), ('font-weight', 'bold')]},
      {'selector': 'th', 'props': [('font-size', '12pt'), ('text-align', 'center'), ('border', '1px solid black')]},
      {'selector': 'td', 'props': [('font-size', '12pt'), ('text-align', 'center'), ('border', '1px solid black')]},
      {'selector': 'table', 'props': [('border-collapse', 'collapse')]}  # Add border-collapse to merge borders
  ])


# Save the DataFrame to an Excel file
result_df.to_excel("results/.xlsx/frm2-aid_rab6-compartment-integrity.xlsx", sheet_name="Table")
